### Optimization of 2-D Himmelblaue function for varied coefficients
##### Reference: https://en.wikipedia.org/wiki/Himmelblau%27s_function
$$ cost(a,b,x,y) =  (x^2+y-a)^2 + (x+y^2-b)^2 ,$$
$$pdf(a,b,x,y) = e^{-cost(a,b,x,y)}$$ 

Here, $\mathbf{x}_{task}=(a,b)$ and $\mathbf{x}_{decision} = (x,y)$

Depending on the choice of task-parameters $(a,b)$ there could be several global optima.

We show that TTGO is able to find the multiple global optima consistently with a hand few of samples from the constructed tt-model of the above pdf (constructed offline) for various selection of $\mathbf{x}_{task}=(a,b)$ in the online phase.  We use scipy's SLSQP to fine tune the initialization. 

Condition on different values of $\mathbf{x}_{task}=(a,b)$ to test the model. Watch out for the multimodality in the solutions of TTGO!

Copyright (c) 2008 Idiap Research Institute, http://www.idiap.ch/
    Written by Suhan Shetty <suhan.shetty@idiap.ch>,


In [ ]:
import torch
import numpy as np
import sys
sys.path.append('./fcn_opt')
sys.path.append('../')

from ttgo import TTGO
import tt_utils
from test_fcns import Himmelblaue_4D 
from fcn_plotting_utils import plot_surf, plot_contour

%load_ext autoreload
np.set_printoptions(precision=3)
%autoreload 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

### Define the function

In [ ]:
pdf, cost =  Himmelblaue_4D(alpha=0.25)

### Define the domain and the discretization

In [ ]:
# Define the domain of the function
L = 5 # [-L,L]^2 is the domain of the function
# domain of task params: domain of coefficients a and b in Himmelblaue 
domain_task = [torch.linspace(1,15,100).to(device)]+[torch.linspace(1,15,500).to(device)] 
# domain of decision variables
domain_decision = [torch.linspace(-L,L,100).to(device)]*2 # domain of x-y coordinates 
domain = domain_task+domain_decision


In [ ]:
# Find the tt-model corresponding to the pdf
tt_model = tt_utils.cross_approximate(fcn=pdf,  domain=domain, 
                        rmax=200, nswp=20, eps=1e-3, verbose=True, 
                        kickrank=5, device=device)

### Fit the TT-Model

In [ ]:
# Refine the discretization and interpolate the model
scale_factor = 20
site_list = torch.arange(len(domain))#len(domain_task)+torch.arange(len(domain_decision))
domain_new = tt_utils.refine_domain(domain=domain, 
                                    site_list=site_list,
                                    scale_factor=scale_factor, device=device)
tt_model_new = tt_utils.refine_model(tt_model=tt_model, 
                                    site_list=site_list,
                                    scale_factor=scale_factor, device=device)

In [ ]:
ttgo = TTGO(tt_model=tt_model_new,domain=domain_new,cost=cost, device=device)

In [ ]:
# torch.save([ttgo.tt_model,domain],'himmel4D.pickle')

### Sample from TT-Model

In [ ]:
a=14; b=2.
x_task = torch.tensor([a,b]).view(1,-1).to(device) #given task-parameters
n_samples_tt = 100
samples = ttgo.sample_tt(n_samples=n_samples_tt, x_task=x_task.view(1,-1), alpha=0.9) 

### Choose the best sample as an estimate for optima

In [ ]:
best_estimate = ttgo.choose_best_sample(samples)[0]
top_k_estimate = ttgo.choose_top_k_sample(samples,k=50)[0] # for multiple solutions

##### Fine-tune the estimate using gradient-based optimization

In [ ]:
ttgo_optimized, _ = ttgo.optimize(best_estimate)

ttgo_optimized_k = 1*top_k_estimate
for i, x in enumerate(ttgo_optimized_k):
    ttgo_optimized_k[i], _ = ttgo.optimize(x)


In [ ]:
print("PDF at the estimated point: ", pdf(best_estimate))
print("PDF at the optima: ", pdf(ttgo_optimized))

In [ ]:
print("Estimated Optima: ", best_estimate)
print("Optima: ", ttgo_optimized)

### Visualization

In [ ]:
# Redefinig the function given the coefficients
def cost_fcn(X):
    X = torch.from_numpy(X)
    X_ext = torch.empty(X.shape[0],4)
    X_ext[:,:2] = x_task
    X_ext[:,2:] = X
    return cost(X_ext.to(device)).cpu().numpy()

In [ ]:
x = np.linspace(-L,L,200)
y = np.linspace(-L,L,200)
data = samples[0,:,2:].cpu()

plt=plot_contour(x,y,cost_fcn,data=data, contour_scale=1000, figsize=10, markersize=1)
# plt.plot(ttgo_optimized[:,2],ttgo_optimized[:,3],'*r',markersize=10)
plt.plot(ttgo_optimized_k[:,2].cpu(),ttgo_optimized_k[:,3].cpu(),'.r',markersize=10)
# plt.legend(["samples","optima"])
# plt.title(r"Himmelblau: $cost=(x^2+y-{})^2+(x+y^2-{})^2$".format(a,b))
# plt.savefig('Himmelblau4D_a13_b5_alpha0_ns1000_k10.png',pad_inches=0.01, dpi=300)
# plt.plot(gott_top_k_estimate[:,2],gott_top_k_estimate[:,3],'*r',markersize=8)